In [1]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np

import qiime2 as q2

%matplotlib inline

data_dir = 'poop_data/Taxonomy'

In [ ]:
! qiime rescript get-silva-data \
    --p-version '138' \
    --p-target 'SSURef_NR99' \
    --p-include-species-labels \
    --o-silva-sequences $data_dir/silva-138-ssu-nr99-seqs.qza \
    --o-silva-taxonomy $data_dir/silva-138-ssu-nr99-tax.qza

In [10]:
#using the cleaned database from w4 exercise due to time saving/computing benefits
#removing sequences that are shorter than certain threshold (based on whether they belong to Archaea, Bacteria or Eukaryota).
#do the numbers also apply for us?
#i steht für input, p parameter, o output.

! qiime rescript filter-seqs-length-by-taxon \
    --i-sequences $data_dir/silva-138-ssu-nr99-seqs-cleaned.qza \
    --i-taxonomy $data_dir/silva-138-ssu-nr99-tax.qza \
    --p-labels Archaea Bacteria Eukaryota \
    --p-min-lens 900 1200 1400 \
    --o-filtered-seqs $data_dir/silva-138-ssu-nr99-seqs-filt.qza \
    --o-discarded-seqs $data_dir/silva-138-ssu-nr99-seqs-discard.qza

Saved FeatureData[Sequence] to: poop_data/Taxonomy/silva-138-ssu-nr99-seqs-filt.qza
Saved FeatureData[Sequence] to: poop_data/Taxonomy/silva-138-ssu-nr99-seqs-discard.qza


As there may be multiple identical sequences sharing same or different taxonomies in the SILVA database, we will also dereplicate our database. We will keep identical sequence records that have differing taxonomies (`uniq` mode).

In [11]:
! qiime rescript dereplicate \
    --i-sequences $data_dir/silva-138-ssu-nr99-seqs-filt.qza  \
    --i-taxa $data_dir/silva-138-ssu-nr99-tax.qza \
    --p-rank-handles 'silva' \
    --p-mode 'uniq' \
    --p-threads 3 \
    --o-dereplicated-sequences $data_dir/silva-138-ssu-nr99-seqs-derep-uniq.qza \
    --o-dereplicated-taxa $data_dir/silva-138-ssu-nr99-tax-derep-uniq.qza

Saved FeatureData[Sequence] to: poop_data/Taxonomy/silva-138-ssu-nr99-seqs-derep-uniq.qza
Saved FeatureData[Taxonomy] to: poop_data/Taxonomy/silva-138-ssu-nr99-tax-derep-uniq.qza




#the forward and reverse primers used in this experiment:

    FWD: GTGYCAGCMGCCGCGGTAA
    REV: GGACTACNVGGGTWTCTAAT


In [12]:
! qiime feature-classifier extract-reads \
    --i-sequences $data_dir/silva-138-ssu-nr99-seqs-derep-uniq.qza \
    --p-f-primer GTGYCAGCMGCCGCGGTAA \
    --p-r-primer GGACTACNVGGGTWTCTAAT \
    --p-n-jobs 3 \
    --p-read-orientation 'forward' \
    --o-reads $data_dir/silva-our-primers.qza

Saved FeatureData[Sequence] to: poop_data/Taxonomy/silva-our-primers.qza


Since we now have significantly shorter sequences than with started with, we need to dereplicate the database again. Conversely, after extraction some unique sequences may point to different taxonomies so we need to handle those too.

In [13]:
! qiime rescript dereplicate \
    --i-sequences $data_dir/silva-our-primers.qza \
    --i-taxa $data_dir/silva-138-ssu-nr99-tax-derep-uniq.qza \
    --p-rank-handles 'silva' \
    --p-mode 'uniq' \
    --p-threads 3 \
    --o-dereplicated-sequences $data_dir/silva-our-primers-derep-uniq.qza \
    --o-dereplicated-taxa  $data_dir/silva-138-ssu-nr99-tax-515f-806r-derep-uniq.qza

Saved FeatureData[Sequence] to: poop_data/Taxonomy/silva-our-primers-derep-uniq.qza
Saved FeatureData[Taxonomy] to: poop_data/Taxonomy/silva-138-ssu-nr99-tax-515f-806r-derep-uniq.qza


**Training classifier:** using pretrained classifier will probably not work if we have to change the primer sequneces

In [ ]:
# ! qiime feature-classifier fit-classifier-naive-bayes \
#     --i-reference-reads $data_dir/silva-our-primers-derep-uniq.qza \
#     --i-reference-taxonomy $data_dir/silva-138-ssu-nr99-tax-515f-806r-derep-uniq.qza \
#     --o-classifier $data_dir/trained-classifier.qza

In [ ]:
#! wget -nv -O $data_dir/515f-806r-classifier.qza https://data.qiime2.org/2021.4/common/gg-13-8-99-515-806-nb-classifier.qza

**Assigning Taxonomy** greengenes is okay or should we use better one? who can run this on their computer?

In [ ]:
! qiime feature-classifier classify-sklearn \
    --i-classifier $data_dir/trained-classifier.qza \
    --i-reads $'poop_data/Denoising'/dada2_rep_set.qza \
    --o-classification $data_dir/taxonomy.qza

In [ ]:
! qiime tools peek $data_dir/taxonomy.qza

**Visualization**

In [ ]:
! qiime metadata tabulate \
    --m-input-file $data_dir/taxonomy.qza \
    --o-visualization $data_dir/taxonomy.qzv

In [ ]:
Visualization.load(f'{data_dir}/taxonomy.qzv')

**filtering out mitochondria and chloroplasts**

In [ ]:
! qiime taxa filter-table \
    --i-table $'poop_data/Denoising'/dada2_table.qza \
    --i-taxonomy $data_dir/taxonomy.qza \
    --p-exclude mitochondria,chloroplast \
    --o-filtered-table $data_dir/table-filtered.qza

! qiime taxa filter-seqs \
    --i-sequences $'poop_data/Denoising'/dada2_rep_set.qza \
    --i-taxonomy $data_dir/taxonomy.qza \
    --p-exclude mitochondria \
    --o-filtered-sequences $data_dir/rep-seqs-filtered.qza

#removes 7 ASVs

In [ ]:
! qiime metadata tabulate \
    --m-input-file $data_dir/rep-seqs-filtered.qza \
    --o-visualization $data_dir/rep-seqs-filtered.qzv

In [ ]:
#vis of filtered sequences. don't have blast links, not the same table
Visualization.load(f'{data_dir}/rep-seqs-filtered.qzv')

In [ ]:
! qiime metadata tabulate \
    --m-input-file $'poop_data'/dada2_rep_set.qza \
    --o-visualization $data_dir/test2.qzv

In [ ]:
Visualization.load(f'{data_dir}/test2.qzv')

In [ ]:
! qiime taxa barplot \
    --i-table $data_dir/table-filtered.qza \
    --i-taxonomy $data_dir/taxonomy.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --o-visualization $data_dir/table-filtered.qzv

In [ ]:
Visualization.load(f'{data_dir}/table-filtered.qzv')

**putting it in Panda**

In [ ]:
pd.set_option('max_colwidth', 150)

In [ ]:
# note: QIIME 2 artifact files can be loaded as python objects! This is how.
taxa = q2.Artifact.load(f'{data_dir}/taxonomy.qza')
# view as a `pandas.DataFrame`. Note: Only some Artifact types can be transformed to DataFrames
taxa = taxa.view(pd.DataFrame)

In [ ]:
taxa.head()

need to do sth to produce change and stage